In [2]:
import os
import sys
import sqlite3
import pandas as pd
import numpy as np
# Spark imports
from pyspark.rdd import RDD
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc
from pyspark import SparkContext, SparkConf

# Create PySpark SparkSession

In [3]:
conf = SparkConf().setAppName('re').setMaster('local')
sc = SparkContext(conf=conf)

In [4]:
spark = SparkSession.builder \
    .master("master") \
    .appName("bigdata-project") \
    .getOrCreate()

# Using pandas to import database

In [6]:
# put all tables of dataset in a dictionary
tables = {}

## Load songs table from database (track_metadata.db)

In [8]:
conection_track_metadata = sqlite3.connect('dataset/track_metadata.db')

- print name of the available tables in the database

In [9]:
cursor = conection_track_metadata.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
track_metadata_tables = cursor.fetchall()
print(track_metadata_tables)

[('songs',)]


In [10]:
tables[track_metadata_tables[0][0]] = pd.read_sql_query("SELECT * FROM songs", conection_track_metadata)
tables['songs']

,track_id,title,song_id,release,artist_id,artist_mbid,artist_name,duration,artist_familiarity,artist_hotttnesss,year,track_7digitalid,shs_perf,shs_work
0,TRMMMYQ128F932D901,Silent Night,SOQMMHC12AB0180CB8,Monster Ballads X-Mas,ARYZTJS1187B98C555,357ff05d-848a-44cf-b608-cb34b5701ae5,Faster Pussy cat,252.05506,0.649822,0.394032,2003,7032331,-1,0
1,TRMMMKD128F425225D,Tanssi vaan,SOVFVAK12A8C1350D9,Karkuteillä,ARMVN3U1187FB3A1EB,8d7ef530-a6fd-4f8f-b2e2-74aec765e0f9,Karkkiautomaatti,156.55138,0.439604,0.356992,1995,1514808,-1,0
2,TRMMMRX128F93187D9,No One Could Ever,SOGTUKN12AB017F4F1,Butter,ARGEKB01187FB50750,3d403d44-36ce-465c-ad43-ae877e65adc4,Hudson Mohawke,138.97098,0.643681,0.437504,2006,6945353,-1,0
3,TRMMMCH128F425532C,Si Vos Querés,SOBNYVR12A8C13558C,De Culo,ARNWYLR1187B9B2F9C,12be7648-7094-495f-90e6-df4189d68615,Yerba Brava,145.05751,0.448501,0.372349,2003,2168257,-1,0
4,TRMMMWA128F426B589,Tangle Of Aspens,SOHSBXH12A8C13B0DF,Rene Ablaze Presents Winter Sessions,AREQDTE1269FB37231,,Der Mystic,514.29832,0.000000,0.000000,0,2264873,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,TRYYYUS12903CD2DF0,O Samba Da Vida,SOTXAME12AB018F136,Pacha V.I.P.,AR7Z4J81187FB3FC59,9d50cb20-7e42-45cc-b0dd-154c3e92a577,Kiko Navarro,217.44281,0.528617,0.411595,0,7522478,-1,0
999996,TRYYYJO128F426DA37,Jago Chhadeo,SOXQYIQ12A8C137FBB,Naale Baba Lassi Pee Gya,ART5FZD1187B9A7FCF,2357c400-9109-42b6-b3fe-9e2d9f8e3872,Kuldeep Manak,244.16608,0.401500,0.374866,0,1632096,-1,0
999997,TRYYYMG128F4260ECA,Novemba,SOHODZI12A8C137BB3,Dub_Connected: electronic music,ARZ3R6M1187B9AF750,8b97e9c8-61f5-4615-9a96-276f24204e34,Gabriel Le Mar,553.03791,0.556918,0.336914,0,2219291,-1,0
999998,TRYYYDJ128F9310A21,Faraday,SOLXGOR12A81C21EB7,The Trance Collection Vol. 2,ARCMCOK1187B9B1073,4ac5f3de-c5ad-475e-ad50-41f1ef9dba20,Elude,484.51873,0.403668,0.256935,0,5472456,-1,0


## Load artist_similarity database: 

### database connections and artist_similarity tables

In [11]:
conection_artist_similarity = sqlite3.connect('dataset/artist_similarity.db')

In [12]:
cursor = conection_artist_similarity.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
artist_similarity_tables = cursor.fetchall()
print(artist_similarity_tables)

[('artists',), ('similarity',)]


In [15]:
artist_similarity_tables[0][0]

'artists'

In [16]:
tables[artist_similarity_tables[0][0]] = pd.read_sql_query("SELECT * FROM artists", conection_artist_similarity)
tables['artists']

,artist_id
0,AR002UA1187B9A637D
1,AR003FB1187B994355
2,AR006821187FB5192B
3,AR009211187B989185
4,AR009SZ1187B9A73F4
...,...
44740,ARZZXT51187FB4627E
44741,ARZZYRB1187B99D0B6
44742,ARZZYRH11C8A416A12
44743,ARZZZAI124207819C9


- similarity table

In [17]:
artist_similarity_tables[1][0]

'similarity'

In [18]:
tables[artist_similarity_tables[1][0]] = pd.read_sql_query("SELECT * FROM similarity", conection_artist_similarity)
tables['similarity']

,target,similar
0,AR002UA1187B9A637D,ARQDOR81187FB3B06C
1,AR002UA1187B9A637D,AROHMXJ1187B989023
2,AR002UA1187B9A637D,ARAGWVR1187B9B749B
3,AR002UA1187B9A637D,AREQVWS1241B9CC0A4
4,AR002UA1187B9A637D,ARHBE351187FB3B0CD
...,...,...
2201911,ARZZZKG1271F573BC4,ARBYTEZ1187FB42EC2
2201912,ARZZZKG1271F573BC4,AR3B4IC1187B98C8B6
2201913,ARZZZKG1271F573BC4,ARM9KFI1187B9B356C
2201914,ARZZZKG1271F573BC4,ARF5QVT1187B9946B9


### Loading 5 tables from artist_term.db (artist_mbtag, artist_term, artists, mbtags, terms): 

- connect to database

In [19]:
connection_artist_term = sqlite3.connect('dataset/artist_term.db')

In [21]:
cursor = connection_artist_term.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
artist_term_tables = cursor.fetchall()
print(artist_term_tables)

[('artists',), ('terms',), ('artist_term',), ('mbtags',), ('artist_mbtag',)]


- Query to "artist_mbtag" table and create pandas dataframe and add it to the table

In [22]:
artist_term_tables[4][0]

'artist_mbtag'

In [23]:
tables[artist_term_tables[4][0]] = pd.read_sql_query("SELECT * FROM artist_mbtag", connection_artist_term)
tables['artist_mbtag']

,artist_id,mbtag
0,AR002UA1187B9A637D,uk
1,AR002UA1187B9A637D,rock
2,AR002UA1187B9A637D,garage rock
3,AR006821187FB5192B,bass
4,AR00A6H1187FB5402A,detroit
...,...,...
24772,ARZZMWO1187B9AFB36,usa
24773,ARZZMWO1187B9AFB36,metal
24774,ARZZXT51187FB4627E,classic pop and rock
24775,ARZZYRB1187B99D0B6,jazz


- Query to "artist_term" table and create pandas dataframe

In [25]:
artist_term_tables[2][0]

'artist_term'

In [26]:
tables[artist_term_tables[2][0]] =  pd.read_sql_query("SELECT * FROM artist_term", connection_artist_term)
tables['artist_term'] 

,artist_id,term
0,AR002UA1187B9A637D,garage rock
1,AR002UA1187B9A637D,country rock
2,AR002UA1187B9A637D,free jazz
3,AR002UA1187B9A637D,oi
4,AR002UA1187B9A637D,space rock
...,...,...
1109376,ARZZZKG1271F573BC4,heavy
1109377,ARZZZKG1271F573BC4,dark
1109378,ARZZZKG1271F573BC4,instrumental
1109379,ARZZZKG1271F573BC4,beautiful


- Query to "mbtags" table and create pandas dataframe

In [27]:
artist_term_tables[3][0]

'mbtags'

In [28]:
tables[artist_term_tables[3][0]] = pd.read_sql_query("SELECT * FROM mbtags", connection_artist_term)
tables['mbtags']

,mbtag
0,00s
1,00s 10s
2,1 13 165900 150 7672 22647 34612 48720 59280 7...
3,1 7 186240 183 23558 41608 89158 111733 150833...
4,10s
...,...
2316,ਭੰਗੜਾ
2317,ハロー プロジェクト
2318,姚姓
2319,日本


- Query to "terms" table and create pandas dataframe

In [29]:
artist_term_tables[1][0]

'terms'

In [32]:
tables[artist_term_tables[1][0]] = pd.read_sql_query("SELECT * FROM terms", connection_artist_term)
tables['terms']

,term
0,00s
1,00s alternative
2,00s country
3,00s indie
4,00s pop
...,...
7638,zxzw 2006
7639,zxzw 2007
7640,zxzw 2008
7641,zxzw 2009


In [33]:
tables.keys()

dict_keys(['songs', 'artists', 'similarity', 'artist_mbtag', 'artist_term', 'mbtags', 'terms'])

In [34]:
type(tables['songs'])

pandas.core.frame.DataFrame

# Convert Pandas dataframe to PySpark DataFrame

In [36]:
for key in tables.keys(): 
    tables[key] = spark.createDataFrame(tables[key]) 

In [38]:
type(tables['songs'])

pyspark.sql.dataframe.DataFrame